<a href="https://colab.research.google.com/github/rohitsahu9820/PRODIGY_DS_04/blob/main/analyze_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install pandas scikit-learn matplotlib seaborn nltk

# Upload your CSV files
from google.colab import files
uploaded = files.upload()


Saving twitter_training.csv to twitter_training.csv
Saving twitter_validation.csv to twitter_validation.csv


In [2]:
import pandas as pd

# Load the files
train_df = pd.read_csv('twitter_training.csv', header=None)
valid_df = pd.read_csv('twitter_validation.csv', header=None)

# Rename columns for clarity
train_df.columns = ['id', 'entity', 'sentiment', 'text']
valid_df.columns = ['id', 'entity', 'sentiment', 'text']


In [3]:
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return text.lower().strip()

train_df['clean_text'] = train_df['text'].apply(clean_text)
valid_df['clean_text'] = valid_df['text'].apply(clean_text)


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['sentiment'])
valid_df['label'] = le.transform(valid_df['sentiment'])


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['clean_text'])
X_valid = vectorizer.transform(valid_df['clean_text'])

# Train model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, train_df['label'])

# Predict
preds = clf.predict(X_valid)
print(classification_report(valid_df['label'], preds, target_names=le.classes_))


              precision    recall  f1-score   support

  Irrelevant       0.76      0.70      0.73       172
    Negative       0.76      0.88      0.81       266
     Neutral       0.87      0.73      0.79       285
    Positive       0.81      0.86      0.83       277

    accuracy                           0.80      1000
   macro avg       0.80      0.79      0.79      1000
weighted avg       0.80      0.80      0.80      1000

